In [1]:
from bs4 import BeautifulSoup, NavigableString
import re
import pickle
import datetime
from numpy import nan

In [27]:
d = pickle.load(open('../data/deputy_webpages_offices_sejm8.pickle', 'rb'))

In [3]:
# html = open('../data/Adam Abramowicz.html', 'r').read()
html = list(d.values())[0]

In [4]:
soup = BeautifulSoup(html, 'lxml')

In [5]:
#Name
soup.find(id='title_content').h1.text

'Sławomir Kłosowski'

In [6]:
#Party
soup.find(id='lblLista').next_sibling.text

'Prawo i Sprawiedliwość'

In [7]:
#Image
soup.find('img').attrs['src']

'http://orka.sejm.gov.pl/Poslowie8.nsf/0/8EC5782903E9ED87C1257EEE00320CB8/$File/161.jpg'

In [8]:
#Birth date
bd = soup.find(id='urodzony').next_element
bd

'21-02-1964'

In [11]:
#Age
(datetime.datetime.now() - datetime.datetime.strptime(bd, '%d-%m-%Y')).days // 365

51

In [12]:
#Birth place
soup.find(id='urodzony').contents[2]

'Wambierzyce'

In [13]:
#Adres głównego biura
x=soup.find('tbody')

In [14]:
street = x.find(string=re.compile('^[\*Biuro]')).next.nobr.text
street

'ul. Pl. Kopernika 1 m. 5'

In [15]:
zip_town = x.find(string=re.compile('^[\*Biuro]')).next.nobr.find_next('nobr').text
zip_town

'45-040 Opole'

In [16]:
area = x.find(string=re.compile('^[\*Biuro]')).next.nobr.find_next('nobr').find_next('nobr').text
area

'woj. opolskie'

In [17]:
#Liczba biur
len(x.find_all('tr'))-1

1

In [18]:
#Telefon do glownego biura
x.find(text=re.compile(r'tel.')).replace('\n', '').replace('tel. ', '')

AttributeError: 'NoneType' object has no attribute 'replace'

In [19]:
#Liczba głosów
soup.find(id='lblGlosy').next_sibling.text

'17048'

In [20]:
#Wykształcenie
soup.find(id='lblWyksztalcenie').next_sibling.text

''

In [21]:
#Szkoły
[x for x in soup.find(id='lblSzkola').next_sibling.children if isinstance(x, NavigableString)]

[]

## Creating a dataset

In [22]:
import pandas

In [28]:
df = pandas.DataFrame(columns=['link', 'name', 'party_list', 'picture', 'date_of_birth', 'birth_place', 'main_office_street', 'main_office_zip_town', 'main_office_region', 'main_office_phone', 'offices', 'votes'])

def parse_page(url, html):
    soup = BeautifulSoup(html, 'lxml')
    result = [url]
    try:
        result += [soup.find(id='title_content').h1.text]
    except:
        result += ['']
        print('failed url:', url)
    try:
        result += [soup.find(id='lblLista').next_sibling.text]
    except:
        result += ['']
        print('failed url:', url)
    result += [soup.find('img').attrs['src']]
    bd = soup.find(id='urodzony').next_element
    result += [datetime.datetime.strptime(bd, '%d-%m-%Y')]
    result += [soup.find(id='urodzony').contents[2]]
    x=soup.find('tbody')
    try:
        result += [x.find(string=re.compile('^[\*Biuro]')).next.nobr.text]
        result += [x.find(string=re.compile('^[\*Biuro]')).next.nobr.find_next('nobr').text]
        result += [x.find(string=re.compile('^[\*Biuro]')).next.nobr.find_next('nobr').find_next('nobr').text]
    except:
        result += ['', '', '']
    try:
        result += [x.find(text=re.compile(r'tel.')).replace('\n', '').replace('tel. ', '')]
    except:
        result += ['']
    try:
        result += [len(x.find_all('tr'))-1]
    except:
        result += [0]
    result += [soup.find(id='lblGlosy').next_sibling.text]
    return result

for i, item in enumerate(d.items()):
    url, html = item
    df.loc[i] = parse_page(url, html)

In [29]:
df.head()

,link,name,party_list,picture,date_of_birth,birth_place,main_office_street,main_office_zip_town,main_office_region,main_office_phone,offices,votes
0,http://www.sejm.gov.pl/Sejm8.nsf/posel.xsp?id=...,Kamila Gasiuk-Pihowicz,Nowoczesna Ryszarda Petru,http://orka.sejm.gov.pl/Poslowie8.nsf/0/5F01A7...,1983-05-08,Warszawa,,,,,-1,19041
1,http://www.sejm.gov.pl/Sejm8.nsf/posel.xsp?id=...,Jan Szyszko,Prawo i Sprawiedliwość,http://orka.sejm.gov.pl/Poslowie8.nsf/0/6B5DAD...,1944-04-19,Stara Miłosna,,,,,-1,15015
2,http://www.sejm.gov.pl/Sejm8.nsf/posel.xsp?id=...,Kornel Morawiecki,Kukiz'15,http://orka.sejm.gov.pl/Poslowie8.nsf/0/4D9A1D...,1941-05-03,Warszawa,ul. Barlickiego 2b,50-324 Wrocław,woj. dolnośląskie,,1,26101
3,http://www.sejm.gov.pl/Sejm8.nsf/posel.xsp?id=...,Marcin Święcicki,Platforma Obywatelska,http://orka.sejm.gov.pl/Poslowie8.nsf/0/9B9F2D...,1947-04-17,Warszawa,ul. Nowy Świat 41a,00-042 Warszawa,woj. mazowieckie,(22) 692-72-27,1,8329
4,http://www.sejm.gov.pl/Sejm8.nsf/posel.xsp?id=...,Robert Winnicki,Kukiz'15,http://orka.sejm.gov.pl/Poslowie8.nsf/0/DCDDBB...,1985-07-18,Zgorzelec,ul. Pl. Solidarności 1/3/5,53-661 Wrocław,woj. dolnośląskie,,1,11802


In [31]:
df.offices.value_counts()

 1    330
-1    118
 2      8
 0      3
 3      1
Name: offices, dtype: int64

In [32]:
df.to_excel('../output/ListaPoslow_sejm8.xlsx')

## Creating an education dataset

In [33]:
#Szkoły
szkoly = [x for x in soup.find(id='lblSzkola').next_sibling.children if isinstance(x, NavigableString)]
szkoly

[]

In [34]:
def parse_education(soup):
    szkoly = [x for x in soup.find(id='lblSzkola').next_sibling.children if isinstance(x, NavigableString)]
    for s in szkoly:
        v = s.split(', ')
        szkola = v[0]
        wydzial = ''
        if len(v) == 3:
            wydzial = v[1]
        kierunek = v[-1]
        try:
            rok = re.findall('(\d+)', kierunek)[0]
        except:
            rok = nan
        yield [szkola, wydzial, kierunek, rok]

In [35]:
dfe = pandas.DataFrame(columns=['name', 'date_of_birth', 'birth_place', 'party_list', 'institution', 'department', 'major', 'year'])
i = 0
for html in d.values():
    soup = BeautifulSoup(html, 'lxml')    
    n = soup.find(id='title_content').h1.text
    bd = soup.find(id='urodzony').next_element
    dob = datetime.datetime.strptime(bd, '%d-%m-%Y')
    bp = soup.find(id='urodzony').contents[2]
    par = soup.find(id='lblLista').next_sibling.text
    for e in parse_education(soup):
        dfe.loc[i] = [n, dob, bp, par] + e
        i += 1

In [36]:
dfe.head()

,name,date_of_birth,birth_place,party_list,institution,department,major,year


In [37]:
dfe.describe()

,name,date_of_birth,birth_place,party_list,institution,department,major,year
count,0,0,0,0,0,0,0,0
unique,0,0,0,0,0,0,0,0


In [285]:
dfe.to_excel('../output/Poslowie_Edukacja_sejm8.xlsx')